In [24]:
# https://docs.python.org/2/library/sqlite3.html helped me a lot as well as 
# youtube tutorials on sqlite3 in python, the first three parts: https://www.youtube.com/watch?v=o-vsdfCBpsU
import sqlite3
import json
from datetime import datetime

sql_transaction = []

# connection objects represents the database
conn = sqlite3.connect('2015-01.db')
c = conn.cursor()

# helper function to create a table
def create_table():
   # c.execute("DROP TABLE IF EXISTS parent_reply") # if we want to start with a whole new table
    c.execute("CREATE TABLE IF NOT EXISTS parent_reply(parent_id TEXT PRIMARY KEY, comment_id TEXT UNIQUE, parent TEXT, comment TEXT, subreddit TEXT, unix INT, score INT)")

# we want one comment to be on one line
def format_data(data):
    data = data.replace('\n',' newlinechar ').replace('\r',' returnchar ').replace('"',"'")
    return data

# here we execute most of our transactions, once we buffered enough of them
def transaction_bldr(sql):
    # we add the global keyword because we want to modify the variable on global scale
    global sql_transaction
    sql_transaction.append(sql)
    # we don't want to be making queries into the database all the time, it is ineffective, so we wait for 1k transactions first
    if len(sql_transaction) > 1000:
        c.execute('BEGIN TRANSACTION')
        for s in sql_transaction:
            try:
                c.execute(s)
            except:
                pass
        conn.commit()
        sql_transaction = []

        
# next functions are just helper functions to perform basic sql operations, updates and inserts        

def sql_insert_replace_comment(commentid, parentid, parent, comment, subreddit, time, score):
    try:
        sql = """UPDATE parent_reply SET parent_id = ?, comment_id = ?, parent = ?, comment = ?, subreddit = ?, 
        unix = ?, score = ? WHERE parent_id =?;""".format(parentid, commentid, parent, comment, subreddit, int(time), score, parentid)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))

def sql_insert_has_parent(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, parent, comment, subreddit, unix, score) 
        VALUES ("{}","{}","{}","{}","{}",{},{});""".format(parentid, commentid, parent, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))

def sql_insert_no_parent(commentid,parentid,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, comment, subreddit, unix, score) 
        VALUES ("{}","{}","{}","{}",{},{});""".format(parentid, commentid, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))


# we define when is comment acceptable. 
def acceptable(data):
    # we dont want comments to be more than 50 words long...(for training purposes)
    if len(data.split(' ')) > 50 or len(data) < 1:
        return False
    # ... or 1k characters
    elif len(data) > 1000:
        return False
    # or if they were deleted or removed by admin
    elif data == '[deleted]': or data == '[removed]'
        return False
    else:
        return True

def find_parent(pid):
    try:
        sql = "SELECT comment FROM parent_reply WHERE comment_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else: return False
    except Exception as e:
        #print(str(e))
        return False

def find_existing_score(pid):
    try:
        sql = "SELECT score FROM parent_reply WHERE parent_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else: return False
    except Exception as e:
        #print(str(e))
        return False
    
if __name__ == '__main__':
    create_table()
    row_counter = 0
    paired_rows = 0

    # since I have whole month of reddit comments, the data has almost 32GB of size, so I process them 1k at a time
    with open('/media/doriot/Elements/RC_2015-01', buffering=1000) as f:
        for row in f:
            row_counter += 1
            # there is many more attributes, but for now I decided to pick the following:
            row = json.loads(row)
            parent_id = row['parent_id']
            body = format_data(row['body'])
            created_utc = row['created_utc']
            score = row['score']
           # comment_id = row['name']
            subreddit = row['subreddit']
            
            parent_data = find_parent(parent_id)
            
            # we only consider comments that have at least score of 2
            if score >= 2:
                # in this section we want to find out whether an existing child of a parent has a lower score 
                # than the current comment. If so, replace it.
                existing_comment_score = find_existing_score(parent_id)
                if existing_comment_score:
                    if score > existing_comment_score:
                        if acceptable(body):
                            sql_insert_replace_comment(comment_id, parent_id, parent_data, body, subreddit, created_utc, score)
                
                # if the parent comment doesn't have a child
                else:
                    if acceptable(body):
                        # and has a parent
                        if parent_data:
                            sql_insert_has_parent(comment_id, parent_id, parent_data, body, subreddit, created_utc, score)
                            paired_rows += 1
                        # and hasn't got a parent
                        else:
                            sql_insert_no_parent(comment_id, parent_id, body, subreddit, created_utc, score)
                            
            # po spracovani kazdych 100k riadkov si urobim kontrolny vypis
            if row_counter % 100000 == 0:
                print('Total Rows Read: {}, Paired Rows: {}, Time: {}'.format(row_counter, paired_rows, str(datetime.now())))

Total Rows Read: 100000, Paired Rows: 3718, Time: 2018-01-14 15:33:07.718606
Total Rows Read: 200000, Paired Rows: 8432, Time: 2018-01-14 15:33:13.514158
Total Rows Read: 300000, Paired Rows: 13772, Time: 2018-01-14 15:33:19.340454
Total Rows Read: 400000, Paired Rows: 19179, Time: 2018-01-14 15:33:25.208208
Total Rows Read: 500000, Paired Rows: 24365, Time: 2018-01-14 15:33:30.714868
Total Rows Read: 600000, Paired Rows: 28878, Time: 2018-01-14 15:33:36.118976
Total Rows Read: 700000, Paired Rows: 33611, Time: 2018-01-14 15:33:42.708454
Total Rows Read: 800000, Paired Rows: 39023, Time: 2018-01-14 15:33:48.351413
Total Rows Read: 900000, Paired Rows: 44662, Time: 2018-01-14 15:33:55.199840
Total Rows Read: 1000000, Paired Rows: 50247, Time: 2018-01-14 15:34:00.836592
Total Rows Read: 1100000, Paired Rows: 55890, Time: 2018-01-14 15:34:06.500946
Total Rows Read: 1200000, Paired Rows: 61380, Time: 2018-01-14 15:34:12.098421
Total Rows Read: 1300000, Paired Rows: 67071, Time: 2018-01-14 

Total Rows Read: 10400000, Paired Rows: 581946, Time: 2018-01-14 15:42:59.385274
Total Rows Read: 10500000, Paired Rows: 587347, Time: 2018-01-14 15:43:05.216800
Total Rows Read: 10600000, Paired Rows: 592208, Time: 2018-01-14 15:43:11.179993
Total Rows Read: 10700000, Paired Rows: 597088, Time: 2018-01-14 15:43:16.978511
Total Rows Read: 10800000, Paired Rows: 602615, Time: 2018-01-14 15:43:22.848012
Total Rows Read: 10900000, Paired Rows: 608336, Time: 2018-01-14 15:43:28.652134
Total Rows Read: 11000000, Paired Rows: 614359, Time: 2018-01-14 15:43:34.430240
Total Rows Read: 11100000, Paired Rows: 620388, Time: 2018-01-14 15:43:40.344460
Total Rows Read: 11200000, Paired Rows: 626342, Time: 2018-01-14 15:43:46.256839
Total Rows Read: 11300000, Paired Rows: 632509, Time: 2018-01-14 15:43:52.208835
Total Rows Read: 11400000, Paired Rows: 638744, Time: 2018-01-14 15:43:58.197401
Total Rows Read: 11500000, Paired Rows: 644898, Time: 2018-01-14 15:44:04.246655
Total Rows Read: 11600000, P

Total Rows Read: 20500000, Paired Rows: 1160208, Time: 2018-01-14 15:53:01.804392
Total Rows Read: 20600000, Paired Rows: 1165505, Time: 2018-01-14 15:53:07.754255
Total Rows Read: 20700000, Paired Rows: 1170811, Time: 2018-01-14 15:53:13.585980
Total Rows Read: 20800000, Paired Rows: 1176029, Time: 2018-01-14 15:53:19.461204
Total Rows Read: 20900000, Paired Rows: 1181544, Time: 2018-01-14 15:53:25.426661
Total Rows Read: 21000000, Paired Rows: 1186670, Time: 2018-01-14 15:53:31.327347
Total Rows Read: 21100000, Paired Rows: 1191269, Time: 2018-01-14 15:53:37.387320
Total Rows Read: 21200000, Paired Rows: 1196455, Time: 2018-01-14 15:53:43.489478
Total Rows Read: 21300000, Paired Rows: 1201958, Time: 2018-01-14 15:53:49.629439
Total Rows Read: 21400000, Paired Rows: 1207853, Time: 2018-01-14 15:53:55.836294
Total Rows Read: 21500000, Paired Rows: 1213852, Time: 2018-01-14 15:54:01.998466
Total Rows Read: 21600000, Paired Rows: 1219847, Time: 2018-01-14 15:54:08.203192
Total Rows Read:

Total Rows Read: 30500000, Paired Rows: 1730588, Time: 2018-01-14 16:03:09.144740
Total Rows Read: 30600000, Paired Rows: 1735576, Time: 2018-01-14 16:03:15.204349
Total Rows Read: 30700000, Paired Rows: 1739831, Time: 2018-01-14 16:03:20.882578
Total Rows Read: 30800000, Paired Rows: 1745121, Time: 2018-01-14 16:03:26.822005
Total Rows Read: 30900000, Paired Rows: 1750750, Time: 2018-01-14 16:03:32.697504
Total Rows Read: 31000000, Paired Rows: 1756090, Time: 2018-01-14 16:03:38.603791
Total Rows Read: 31100000, Paired Rows: 1762080, Time: 2018-01-14 16:03:44.712428
Total Rows Read: 31200000, Paired Rows: 1767978, Time: 2018-01-14 16:03:50.816947
Total Rows Read: 31300000, Paired Rows: 1773884, Time: 2018-01-14 16:03:56.942842
Total Rows Read: 31400000, Paired Rows: 1779396, Time: 2018-01-14 16:04:03.059640
Total Rows Read: 31500000, Paired Rows: 1784217, Time: 2018-01-14 16:04:09.027983
Total Rows Read: 31600000, Paired Rows: 1789383, Time: 2018-01-14 16:04:15.095378
Total Rows Read:

Total Rows Read: 40500000, Paired Rows: 2298855, Time: 2018-01-14 16:13:15.167641
Total Rows Read: 40600000, Paired Rows: 2303984, Time: 2018-01-14 16:13:21.166063
Total Rows Read: 40700000, Paired Rows: 2308817, Time: 2018-01-14 16:13:27.108749
Total Rows Read: 40800000, Paired Rows: 2314096, Time: 2018-01-14 16:13:33.140336
Total Rows Read: 40900000, Paired Rows: 2319743, Time: 2018-01-14 16:13:39.215842
Total Rows Read: 41000000, Paired Rows: 2325507, Time: 2018-01-14 16:13:45.335374
Total Rows Read: 41100000, Paired Rows: 2331486, Time: 2018-01-14 16:13:51.432361
Total Rows Read: 41200000, Paired Rows: 2337532, Time: 2018-01-14 16:13:57.536135
Total Rows Read: 41300000, Paired Rows: 2343523, Time: 2018-01-14 16:14:03.636913
Total Rows Read: 41400000, Paired Rows: 2349484, Time: 2018-01-14 16:14:09.707308
Total Rows Read: 41500000, Paired Rows: 2355254, Time: 2018-01-14 16:14:15.697690
Total Rows Read: 41600000, Paired Rows: 2360807, Time: 2018-01-14 16:14:21.628516
Total Rows Read:

Total Rows Read: 50500000, Paired Rows: 2873955, Time: 2018-01-14 16:23:15.889750
Total Rows Read: 50600000, Paired Rows: 2879840, Time: 2018-01-14 16:23:21.900253
Total Rows Read: 50700000, Paired Rows: 2885631, Time: 2018-01-14 16:23:28.075629
Total Rows Read: 50800000, Paired Rows: 2891545, Time: 2018-01-14 16:23:34.076631
Total Rows Read: 50900000, Paired Rows: 2897601, Time: 2018-01-14 16:23:40.058823
Total Rows Read: 51000000, Paired Rows: 2903345, Time: 2018-01-14 16:23:46.186232
Total Rows Read: 51100000, Paired Rows: 2908718, Time: 2018-01-14 16:23:52.100947
Total Rows Read: 51200000, Paired Rows: 2913561, Time: 2018-01-14 16:23:58.182087
Total Rows Read: 51300000, Paired Rows: 2918723, Time: 2018-01-14 16:24:04.111518
Total Rows Read: 51400000, Paired Rows: 2924328, Time: 2018-01-14 16:24:10.055669
Total Rows Read: 51500000, Paired Rows: 2930094, Time: 2018-01-14 16:24:16.240441
Total Rows Read: 51600000, Paired Rows: 2936110, Time: 2018-01-14 16:24:22.286972
Total Rows Read:

In [28]:
# here we prepare data for nmt-chatbot
# https://github.com/daniel-kukiela/nmt-chatbot example of how should data look so they can be run with nmtchat
import pandas as pd

# helper function to create files that will be used in training and testing
def write_to_file(fname):
    with open(fname, 'a', encoding='utf8') as f:
        for comment in df[fname.split('_')[0]].values:
            f.write(comment+'\n')
    
# we connect to our local database
db = sqlite3.connect('2015-01.db')
#c = conn.cursor()
batch_size = 10000
last_unix = 0
curr_length = batch_size
i = 0
test_finished = False

while curr_length == batch_size:

    df = pd.read_sql("SELECT * FROM parent_reply WHERE unix > {} and parent NOT NULL and score > 0 ORDER BY unix ASC LIMIT {}".format(last_unix, batch_size), db)
    last_unix = df.tail(1)['unix'].values[0]
    curr_length = len(df)

    # we create test and training sets
    if not test_finished:
        write_to_file('parent_test')
        write_to_file('comment_test')
        test_finished = True
    else:
        write_to_file('parent_train')
        write_to_file('comment_train')
        
        
    if i % 10 == 10:
        print(i * batch_size + curr_length, 'rows completed so far')
    i += 1
        
print('FINISHED')

200000 rows completed so far
400000 rows completed so far
600000 rows completed so far
800000 rows completed so far
1000000 rows completed so far
1200000 rows completed so far
1400000 rows completed so far
1600000 rows completed so far
1800000 rows completed so far
2000000 rows completed so far
2200000 rows completed so far
2400000 rows completed so far
2600000 rows completed so far
2800000 rows completed so far
3000000 rows completed so far


In [26]:
# only checking whether I provided a correct path
# source: https://stackoverflow.com/questions/6046038/open-file-on-separate-drive-in-python

import os
import sys

path = '/media/doriot/Elements/RC_2015-01'
basepath, fname = os.path.split(path)
print("directory:", basepath)

if os.path.exists(basepath):
    print("directory exists")
else:
    print("directory does not exist!")
    sys.exit()

if not fname:
    print("no filename provided!")
    sys.exit()
    
print("filename:", fname)
if os.path.exists(path):
    print("filename exists")
else:
    print("filename not found!")
    print("directory contents:")
    for fn in os.listdir(basepath):
        print(fn)

directory: /media/doriot/Elements
directory does not exist!


SystemExit: 

/home/doriot/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
